In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import matplotlib.pyplot as plt 
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib.font_manager as fm

# jupyter notebook 내 그래프를 바로 그리기 위한 설정
%matplotlib inline

plt.rc('font', family='malgun gothic') # 그래프에서 한글 깨지는 현상 방지
plt.rcParams['axes.unicode_minus']=False #음수 부호 깨짐 방지

In [139]:
playlist = pd.read_csv('./z_save/total_playlist.csv')

## 전처리

In [140]:
playlist['Lyric'] = playlist['Lyric'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
playlist[:5]
playlist['Lyric'] = playlist['Lyric'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
playlist['Lyric'].replace('', np.nan, inplace=True)
print(playlist.isnull().sum())
playlist = playlist.dropna(how = 'any')
playlist.reset_index(drop=True)
print('전처리 후 데이터의 개수 :',len(playlist))

song_id          0
song_name        0
artist           0
album            1
Like_Count       9
Lyric         1122
cover_url       21
tags            21
dtype: int64
전처리 후 데이터의 개수 : 26793


In [141]:
playlist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26793 entries, 0 to 27926
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     26793 non-null  int64 
 1   song_name   26793 non-null  object
 2   artist      26793 non-null  object
 3   album       26793 non-null  object
 4   Like_Count  26793 non-null  object
 5   Lyric       26793 non-null  object
 6   cover_url   26793 non-null  object
 7   tags        26793 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.8+ MB


---

## SentencePiece

> https://inhyeokyoo.github.io/nlp/Sentencepiece/#sentencepiece

In [191]:
import os
import gzip
import pandas as pd
import sentencepiece as spm
import shutil
import csv

학습에 필요한 parameter

spm_train --input=<input> --model_prefix=<model_name> --vocab_size=8000 --character_coverage=1.0 --model_type=<type>

In [ ]:
corpus = "lyric.txt"
prefix = "lyric"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의

> https://paul-hyun.github.io/vocab-with-sentencepiece/ <br>
https://github.com/paul-hyun/transformer-evolution/blob/master/tutorial/vocab_with_sentencepiece.ipynb <br>
https://paul-hyun.github.io/nlp-tutorial-02-02-tokenizer/

* input: 입력 corpus
* prefix: 저장할 모델 이름
* vocab_size: vocab 개수 (기본 8,000에 스페셜 토큰 7개를 더해서 8,007개)
* max_sentence_length: 문장의 최대 길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값
* user_defined_symbols: 사용자 정의 토큰

* --input: 한 라인 당 한 문장을 갖는 corpus로 raw file. Tokenizer, normalizer, preprocessor를 실행할 필요가 없음. default로 Unicode NFKC로 normalize 함. comma-separated로 파일 리스트를 넣을 수 있음.
* --model_prefix: 모델 이름에 붙는 prefix. <model_name>.model와 <model_name>.vocab가 생성.
* --vocab_size: vocabulary size, e.g., 8000, 16000, or 32000
* --character_coverage: 모델에 의해 cover할 character의 양. 일어나 중어 같은 rich character set은 0.9995가 좋은 결과를 내고, 그 외 character set이 작은 언어는 1.0이 좋음.
* --model_type: 모델 타입. unigram (default), bpe, char, or word 중 선택. word을 선택할 경우 반드시 pretokenized 되야 함.


templates에다가 parameter를 입력

In [185]:
templates= '--input={} \
--pad_id={} \
--bos_id={} \
--eos_id={} \
--unk_id={} \
--model_prefix={} \
--vocab_size={} \
--character_coverage={} \
--model_type={}'

In [187]:
lyric = playlist['Lyric']

In [193]:
len(lyric)

26793

In [194]:
# 26793개의 샘플을 sad_lyric.txt 파일에 저장한 후에 센텐스피스를 통해 단어 집합을 생성

with open('lyric.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(playlist['Lyric']))

In [199]:
train_input_file = 'lyric.txt'
pad_id = 0  # <pad>
vocab_size = 2000 # vocab size
prefix = 'lyric_spm'
bos_id = 1 # <start> token
eos_id = 2 # <end> token
unk_id = 3 # <unknown> token
character_coverage = 1.0 # 보통 1
model_type = 'unigram' # Choose from unigram (default), bpe, char, or word

In [200]:
cmd = templates.format(train_input_file,
                pad_id,
                bos_id,
                eos_id,
                unk_id,
                prefix,
                vocab_size,
                character_coverage,
                model_type)

cmd        

'--input=lyric.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=lyric_spm --vocab_size=2000 --character_coverage=1.0 --model_type=unigram'

SentencePieceTrainer의 Train 클래스를 통해 학습할 수가 있다. 이를 통해 생성되는 것은 model과 vocab이며, 앞서 model_prefix를 통해 이름을 지정한 그대로 나오게된다.

In [201]:
spm.SentencePieceTrainer.Train(cmd)

# lyric_spm.model
# lyric_spm.vocab

In [203]:
lyric[0]

'거긴 날씨가 어때   나의 하늘과 밤엔   자꾸 떠오르는 기억이   계속 내게 말을 건네네     우리는 함께일 때   참 많이도 웃었네   지금 나는 무표정을 해  너가 없이 무슨 말을 해      도대체 왜   어떤 걸로도 대신할 수 없는  말로는 설명할 수 없는  무슨 기분일까   무슨 마음일까  내게 왜 그래   나는 왜 이래     보고싶어   너가 보고싶어서   내가 이래 나는 이래   보고싶어   이렇게 보고 싶으면   그건 사랑이래   이건 사랑이네    나는 매일 네 생각을 해   그걸 멈추는게 잘 안돼             이제 나는 네게 말을 해   나의 맘이 흘러 넘치게              '

In [211]:
sp = spm.SentencePieceProcessor()
sp.Load('lyric_spm.model')

sp.SetEncodeExtraOptions('bos:eos') # <s>, </s>를 자동으로 넣어줌

text = '나의 하늘과 밤엔 자꾸 떠오르는 기억이 계속 내게 말을 건네네'

# 문자열을 token으로 분할하는 함수 (encode_as_pieces)
tokens = sp.EncodeAsPieces(text)
print(tokens)

# token을 숫자로 변환하는 함수 (piece_to_id)
ids = sp.EncodeAsIds(text)
print(ids)

['<s>', '▁나의', '▁하늘', '과', '▁밤', '엔', '▁자꾸', '▁', '떠', '오', '르', '는', '▁기억', '이', '▁', '계', '속', '▁내게', '▁말', '을', '▁건', '네', '네', '</s>']
[1, 79, 224, 151, 87, 140, 272, 4, 721, 165, 201, 10, 169, 5, 4, 401, 400, 75, 28, 9, 102, 59, 59, 2]


Detokenize는 DecodePieces와 DecodeIds를 통해 할 수 있다.

token을 문자열로 복원하는 함수 (decode_pieces)

In [208]:
# token을 문자열로 복원
sp.DecodePieces(tokens)


'나의 하늘과 밤엔 자꾸 떠오르는 기억이 계속 내게 말을 건네네'

문자열을 숫자로 분할하는 함수 (encode_as_ids)

In [209]:
sp.DecodeIds(ids)

'나의 하늘과 밤엔 자꾸 떠오르는 기억이 계속 내게 말을 건네네'

## SentencePeice with Morph

In [219]:
import zipfile
import torch
import argparse

In [215]:
lyric_file = 'lyric.txt'

In [220]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234,
    # CPU 또는 GPU 사용여부 결정
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    # 말뭉치 파일
    "corpus": lyric_file,
}
args = argparse.Namespace(**args)

print(args)

Namespace(corpus='lyric.txt', device=device(type='cpu'), seed=1234)


In [226]:
# morph 단위로 분할된 말뭉치 생성
with open("lyric-okt.txt", "w") as o_f:
    # with zipfile.ZipFile(args.corpus) as z:
    with open('lyric.txt',encoding='UTF8') as f:
        for i, line in enumerate(f):
            line = line.strip()
            tokens = okt.morphs(line)
            string = " ".join(tokens)
            o_f.write(string)
            o_f.write("\n")

## Error
UnicodeDecodeError: 'cp949' codec can't decode byte 0xeb in position 7
*  with open('lyric.txt',encoding='UTF8') as f:

## Error
AttributeError: 'str' object has no attribute 'decode'
> https://stackoverflow.com/questions/28583565/str-object-has-no-attribute-decode-python-3-error

In [233]:
def train_sentencepiece(corpus, prefix, vocab_size=1700):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6
        " --input_sentence_size=100000" +  # 말뭉치에서 셈플링해서 학습
        " --shuffle_input_sentence=true")  # 셈플링한 말뭉치 shuffle


In [234]:
# morph vocab 학습
train_sentencepiece("lyric-okt.txt", "lyric_okt_32000")

## Error
RuntimeError: Internal: C:\projects\sentencepiece\src\trainer_interface.cc(590)
Vocabulary size too high (32007). Please set it to a value <= `1800.`
> https://blog.naver.com/yse00800/221972711068

In [248]:
# 생성파일 백업
corpus_dir = os.path.dirname(args.corpus)
shutil.copy("lyric_okt_32000.model", corpus_dir)
shutil.copy("lyric_okt_32000.vocab", corpus_dir)

print(os.listdir(corpus_dir))

FileNotFoundError: [Errno 2] No such file or directory: ''

## Error
* FileNotFoundError: [Errno 2] No such file or directory: ''
* SameFileError: 'lyric_okt_32000.model' and '.\\lyric_okt_32000.model' are the same file

In [250]:
# load morph vocab
spm_morph_vocab = spm.SentencePieceProcessor()
spm_morph_vocab.load(os.path.join(corpus_dir, "lyric_okt_32000.model"))

True

In [252]:
# wiki spm morph tokenize
#with zipfile.ZipFile(args.corpus) as z:
with open('lyric-okt.txt') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        line = line.strip()
        print(line)
        string = " ".join(okt.morphs(line))
        print(string)
        tokens = spm_morph_vocab.encode_as_pieces(string)
        print(tokens)
        _ids = spm_morph_vocab.encode_as_ids(string)
        print(_ids)

거 긴 날씨 가 어때 나 의 하늘 과 밤 엔 자꾸 떠오르는 기억 이 계속 내게 말 을 건네네 우리 는 함께 일 때 참 많이 도 웃었네 지금 나 는 무표정 을 해 너 가 없이 무슨 말 을 해 도대체 왜 어떤 걸 로 도 대신 할 수 없는 말 로는 설명 할 수 없는 무슨 기분 일까 무슨 마음 일까 내게 왜 그래 나 는 왜 이래 보고싶어 너 가 보고싶어서 내 가 이래 나 는 이래 보고싶어 이렇게 보고 싶으면 그건 사랑 이래 이건 사랑 이네 나 는 매일 네 생각 을 해 그걸 멈추는게 잘 안 돼 이제 나 는 네 게 말 을 해 나 의 맘 이 흘러 넘치게
거 긴 날씨 가 어때 나 의 하늘 과 밤 엔 자꾸 떠오르는 기억 이 계속 내게 말 을 건네네 우리 는 함께 일 때 참 많이 도 웃었네 지금 나 는 무표정 을 해 너 가 없이 무슨 말 을 해 도대체 왜 어떤 걸 로 도 대신 할 수 없는 말 로는 설명 할 수 없는 무슨 기분 일까 무슨 마음 일까 내게 왜 그래 나 는 왜 이래 보고싶어 너 가 보고싶어서 내 가 이래 나 는 이래 보고싶어 이렇게 보고 싶으면 그건 사랑 이래 이건 사랑 이네 나 는 매일 네 생각 을 해 그걸 멈추는게 잘 안 돼 이제 나 는 네 게 말 을 해 나 의 맘 이 흘러 넘치게
['▁', '거', '▁', '긴', '▁', '날씨', '▁', '가', '▁', '어때', '▁', '나', '▁', '의', '▁', '하늘', '▁', '과', '▁', '밤', '▁', '엔', '▁', '자꾸', '▁', '떠오르는', '▁', '기', '억', '▁', '이', '▁', '계속', '▁', '내게', '▁', '말', '▁', '을', '▁', '건네네', '▁', '우리', '▁', '는', '▁', '함께', '▁', '일', '▁', '때', '▁', '참', '▁', '많이', '▁', '도', '▁', '웃었네', '▁', '지금', '▁', '나', '▁', '는', '▁', '무표정', '▁', '을', '▁', '해', '▁', '너', '▁'